In [1]:
# import libraries
import pandas as pd
import random

In [7]:
# import data
df = pd.read_excel('../data/bridges.xlsx')

# slice data information
df = df[["road", "km", "type", "name", "length", "condition", "lat", "lon", "zone"]]


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20415 entries, 0 to 20414
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   road       20415 non-null  object 
 1   km         20415 non-null  float64
 2   type       20415 non-null  object 
 3   name       20100 non-null  object 
 4   length     20406 non-null  float64
 5   condition  20415 non-null  object 
 6   lat        20415 non-null  float64
 7   lon        20415 non-null  float64
 8   zone       20415 non-null  object 
dtypes: float64(4), object(5)
memory usage: 1.4+ MB


In [5]:
# HANDLING MISSING VALUES

# change NaN values in name with dot i.e. '.'
df['name'].fillna('.', inplace=True)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20415 entries, 0 to 20414
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   road       20415 non-null  object 
 1   km         20415 non-null  float64
 2   type       20415 non-null  object 
 3   name       20415 non-null  object 
 4   length     20406 non-null  float64
 5   condition  20415 non-null  object 
 6   lat        20415 non-null  float64
 7   lon        20415 non-null  float64
 8   zone       20415 non-null  object 
dtypes: float64(4), object(5)
memory usage: 1.4+ MB


In [2]:

# check NaN values in other columns, only length has missing values
df.isnull().sum(axis=0)

# assign new dataframe to missing values
missing = df[df.length.isnull()]

In [4]:
missing

,road,km,type,name,length,condition,lat,lon,zone
526,N2,64.333,RCC Girder Bridge,Narayanpur Bridge,NaN,A,24.054769,90.921984,Dhaka
1049,N5,27.452,Steel Beam & RCC Slab,Dhulipitha Bridge,NaN,A,23.907829,90.208758,Dhaka
3136,R312,10.992,RCC Girder Bridge,Razabari Bridge-1,NaN,A,24.107651,90.503342,Dhaka
6362,Z1216,7.450,PC Girder Bridge,Kadda Bridge,NaN,A,23.879863,91.199317,Comilla
8560,Z4101,1.340,RCC Girder Bridge,Banabazar Bridge,NaN,A,24.921777,89.961370,Mymensingh
9433,Z5063,10.755,PC Girder Bridge,Purbo Kushtia bridge,NaN,A,23.982234,90.037573,Dhaka
17426,Z1089,2.433,RCC Bridge,Uddabganj Bazar Bridge,NaN,C,23.646071,90.615167,Dhaka
17596,Z1430,19.910,RCC Girder Bridge,Boyoskaler pul Bridge,NaN,C,23.166014,90.961397,Comilla
18278,Z5063,9.215,Truss with Steel Deck,Saturia Bazar Bridge,NaN,C,23.969187,90.036868,Dhaka


In [9]:
# for some missing values, missing values can be retrieved from bridges with same chainage, 
# get length of these bridges and replace missing value with this value. 
for index in missing.index:
    if df.loc[index, 'km'] == df.loc[index - 1, 'km']:
        # assign length of bridge with same chainage to variable new_length
        new_length = df.loc[index - 1, 'length']
        # replace missing value with new length
        df.loc[index, 'length'] = new_length

    elif df.loc[index, 'km'] == df.loc[index + 1, 'km']:
        new_length = df.loc[index + 1, 'length']
        df.loc[index, 'length'] = new_length

# for the left-over missing values, replace length with average length of bridges for specific road
for index in missing.index:
    road_name = df.loc[index, 'road']
    road_subset = df[df['road'] == road_name]
    average_length = road_subset.loc[:, 'length'].mean()
    df.loc[index, 'length'] = average_length

In [10]:

# HANDLING DUPLICATES

# change type of column first
df['name'] = df['name'].astype(str)

# replace modifications of right/left in bridge names
df['name'] = df['name'].apply(lambda x: x.replace(')', ''))
df['name'] = df['name'].apply(lambda x: x.replace('RIGHT', 'R'))
df['name'] = df['name'].apply(lambda x: x.replace('LEFT', 'L'))
df['name'] = df['name'].apply(lambda x: x.replace('Right', 'R'))
df['name'] = df['name'].apply(lambda x: x.replace('Left', 'L'))

# strip the trailing whitespaces 
df['name'] = df['name'].apply(lambda x: x.strip())

# change condition from letters to numbers in order to compare them
df['conditionNum'] = 0
df.loc[df['condition'] == 'A', 'conditionNum'] = 1
df.loc[df['condition'] == 'B', 'conditionNum'] = 2
df.loc[df['condition'] == 'C', 'conditionNum'] = 3
df.loc[df['condition'] == 'D', 'conditionNum'] = 4

In [15]:
# retrieve all roads in dataset
roads = df['road'].unique().tolist()

for road in roads:
    # define dataframe for duplicates
    road_subset = df[df['road'] == road]
    # subset based on latitude and longitude
    duplicates = road_subset[road_subset.duplicated(subset=['lat', 'lon'])]
    # sort by chainage
    duplicates = duplicates.sort_values(by=['km'])
    # retrieve number of duplicates
    duplicates_value = len(duplicates.index)
    if duplicates_value > 50: 
        # print duplicates per road
        print("road", road, "duplicates:", duplicates_value)

road N1 duplicates: 159
road N2 duplicates: 96
road N5 duplicates: 106
road N8 duplicates: 57
